In [26]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/newdataset1/xtrain.csv
/kaggle/input/newdataset1/xtest.csv
/kaggle/input/newdataset1/ytrain.csv
/kaggle/input/newdataset1/ytest.csv


# 1. Random Forest Classifier

In [27]:
# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
from sklearn.metrics import classification_report

# Tree Visualization
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz

In [28]:
X_train = pd.read_csv('/kaggle/input/newdataset1/xtrain.csv')
X_test = pd.read_csv('/kaggle/input/newdataset1/xtest.csv')
y_train = pd.read_csv('/kaggle/input/newdataset1/ytrain.csv')
y_test = pd.read_csv('/kaggle/input/newdataset1/ytest.csv')

In [29]:
X_train.head(5)

,age,gender,height_cm,weight_kg,bmi,duration_minutes,intensity,calories_burned,daily_steps,avg_heart_rate,...,sleep_hours,stress_level,hydration_level,smoking_status,fitness_level,delta_heart_rate,phys_stress_indicator,caloric_efficiency,max_heart_rate,vo2_max
0,-0.264229,0.911589,1.029645,0.903012,0.056218,-1.554278,0.160985,-0.916626,-0.347346,0.021060,...,0.674344,0.264724,-1.715728,-1.616026,-1.749504,-0.000550,0.273775,-0.079731,-0.083348,-0.702112
1,-0.264229,0.911589,1.029645,0.776755,-0.114490,-1.302062,1.592867,-0.633855,-1.102523,1.604136,...,-0.557719,-1.291994,0.180954,-1.616026,-1.734446,1.509959,-1.267376,0.412581,-0.083348,-0.672479
2,-0.264229,0.911589,1.029645,0.941860,0.107947,-1.539947,1.592867,-0.798144,-0.143785,1.660675,...,0.879687,0.264724,-0.163897,-1.616026,-1.711860,1.563906,0.287933,0.454731,-0.083348,-0.711230
3,-0.264229,0.911589,1.029645,0.329998,-0.719725,-0.468030,-1.270897,-0.307713,0.979462,-1.166247,...,-1.584438,-0.773088,0.525806,-1.616026,-1.691156,-1.133432,-0.879670,-0.067450,-0.083348,-0.567625
4,-0.264229,0.911589,1.029645,0.882374,0.030353,-1.261937,1.592867,-0.596457,0.954566,0.529906,...,-2.097797,1.302536,-0.336323,-1.616026,-1.655394,0.484971,1.276195,0.439545,-0.083348,-0.697268


## 1.1. Train RF before Feature Selection 

In [30]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

/tmp/ipykernel_36/2538021705.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


RandomForestClassifier()

In [31]:
rf_y_pred = rf.predict(X_test)

In [32]:
# Evaluation Metrics

rf_accuracy = accuracy_score(y_test, rf_y_pred)
rf_precision = precision_score(y_test, rf_y_pred, average = 'macro')
rf_recall = recall_score(y_test, rf_y_pred, average = 'macro')
rf_f1_score = f1_score(y_test, rf_y_pred, average = 'macro')

print("Accuracy (Before FS): ", rf_accuracy)
print("Precision (Before FS): ", rf_precision)
print("Recall (Before FS): ", rf_recall)
print("F1-Score (Before FS): ", rf_f1_score)

print(classification_report(y_test, rf_y_pred, digits=4))

Accuracy (Before FS):  0.841640378548896
Precision (Before FS):  0.8404585708708716
Recall (Before FS):  0.8356334225247242
F1-Score (Before FS):  0.8369993235092769
              precision    recall  f1-score   support

           1     0.8253    0.9174    0.8689       242
           2     0.8087    0.8819    0.8437       254
           3     0.7027    0.6842    0.6933       190
           4     0.9404    0.8840    0.9113       250
           5     0.9005    0.8190    0.8579       210
           6     0.8525    0.8447    0.8486       219
           7     0.8531    0.8182    0.8353       220

    accuracy                         0.8416      1585
   macro avg     0.8405    0.8356    0.8370      1585
weighted avg     0.8437    0.8416    0.8416      1585



In [33]:
# Get F1-scores per class

rf_f1_scores = {label: metrics['f1-score'] 
             for label, metrics in report.items() if label.isdigit()}

for cls, score in rf_f1_scores.items():
    print(f"{cls}: {score:.4f}")

1: 0.8934
2: 0.8204
3: 0.6987
4: 0.9098
5: 0.8195
6: 0.8227
7: 0.7991


In [34]:
# get the columns from bGA's evaluation [4.7,22]

X_train_fs = X_train.iloc[:, [3, 6, 21]]
X_test_fs = X_test.iloc[:, [3, 6, 21]]
y_train_fs = y_train.copy()
y_test_fs = y_test.copy()

In [35]:
X_train_fs.head()

,weight_kg,intensity,caloric_efficiency
0,0.903012,0.160985,-0.079731
1,0.776755,1.592867,0.412581
2,0.941860,1.592867,0.454731
3,0.329998,-1.270897,-0.067450
4,0.882374,1.592867,0.439545


In [36]:
X_test_fs.head()

,weight_kg,intensity,caloric_efficiency
0,0.974639,-1.270897,-0.330575
1,0.045920,-1.270897,-0.478756
2,-0.081552,-1.270897,-0.845376
3,0.788895,0.160985,-0.360211
4,0.779183,0.160985,-0.542585


## 1.2. Training RF After Feature Selection

In [37]:
rf_fs = RandomForestClassifier()
rf_fs.fit(X_train_fs, y_train_fs)

/tmp/ipykernel_36/3471313091.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_fs.fit(X_train_fs, y_train_fs)


RandomForestClassifier()

In [38]:
rf_y_pred_fs = rf_fs.predict(X_test_fs)

In [39]:
rf_fs_accuracy = accuracy_score(y_test_fs, rf_y_pred_fs)
rf_fs_precision = precision_score(y_test_fs, rf_y_pred_fs, average = 'macro')
rf_fs_recall = recall_score(y_test_fs, rf_y_pred_fs, average = 'macro')
rf_fs_f1_score = f1_score(y_test_fs, rf_y_pred_fs, average = 'macro')

print("Accuracy (After FS): ", rf_fs_accuracy)
print("Precision (After FS): ", rf_fs_precision)
print("Recall (After FS): ", rf_fs_recall)
print("F1-Score (After FS): ", rf_fs_f1_score)

print(classification_report(y_test, rf_y_pred_fs, digits=4))

Accuracy (After FS):  0.9678233438485805
Precision (After FS):  0.965140879191691
Recall (After FS):  0.9645379221184626
F1-Score (After FS):  0.9647866694691881
              precision    recall  f1-score   support

           1     1.0000    0.9959    0.9979       242
           2     0.9805    0.9921    0.9863       254
           3     0.8967    0.8684    0.8824       190
           4     1.0000    0.9960    0.9980       250
           5     0.9904    0.9857    0.9881       210
           6     0.9910    1.0000    0.9955       219
           7     0.8973    0.9136    0.9054       220

    accuracy                         0.9678      1585
   macro avg     0.9651    0.9645    0.9648      1585
weighted avg     0.9677    0.9678    0.9677      1585



# 2. XGBoost Classifier

In [40]:
import xgboost as xgb

## 2.1. Train XGBoost before Feature Selection

In [41]:
# Train XGBoost classifier on all classes

xgb_model = xgb.XGBClassifier()
# subtract 1 from y_train and y_test for xgb

y_train_encoded = y_train - 1
y_test_encoded = y_test - 1

xgb_model.fit(X_train, y_train_encoded)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [42]:
y_train_encoded.head()

,activity_type
0,0
1,0
2,0
3,3
4,0


In [43]:
y_test_encoded.head()

,activity_type
0,0
1,0
2,6
3,6
4,5


In [44]:
xgb_y_pred = xgb_model.predict(X_test)

In [45]:
xgb_accuracy = accuracy_score(y_test_encoded, xgb_y_pred)
xgb_precision = precision_score(y_test_encoded, xgb_y_pred, average = 'macro')
xgb_recall = recall_score(y_test_encoded, xgb_y_pred, average = 'macro')
xgb_f1_score = f1_score(y_test_encoded, xgb_y_pred, average = 'macro')

print("=========== Evaluation for XGBoost ===========")
print("Accuracy (Before FS): ", xgb_accuracy)
print("Precision (Before FS): ", xgb_precision)
print("Recall (Before FS): ", xgb_recall)
print("F1-Score (Before FS): ", xgb_f1_score)
print(classification_report(y_test_encoded, xgb_y_pred, digits=4))

=========== Evaluation for XGBoost ===========
Accuracy (Before FS):  0.9545741324921135
Precision (Before FS):  0.9528027741901007
Recall (Before FS):  0.9518200207283679
F1-Score (Before FS):  0.9521421242968815
              precision    recall  f1-score   support

           0     0.9876    0.9876    0.9876       242
           1     0.9318    0.9685    0.9498       254
           2     0.8821    0.9053    0.8935       190
           3     0.9960    0.9960    0.9960       250
           4     0.9650    0.9190    0.9415       210
           5     0.9909    0.9909    0.9909       219
           6     0.9163    0.8955    0.9057       220

    accuracy                         0.9546      1585
   macro avg     0.9528    0.9518    0.9521      1585
weighted avg     0.9549    0.9546    0.9546      1585



## 2.2. Train XGBoost after Feature Selection

In [46]:
xgb_model_fs = xgb.XGBClassifier()
# subtract 1 from y_train and y_test for xgb

xgb_model_fs.fit(X_train_fs, y_train_encoded)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [47]:
xgb_y_pred_fs = xgb_model_fs.predict(X_test_fs)

In [48]:
xgb_accuracy_fs = accuracy_score(y_test_encoded, xgb_y_pred_fs)
xgb_precision_fs = precision_score(y_test_encoded, xgb_y_pred_fs, average = 'macro')
xgb_recall_fs = recall_score(y_test_encoded, xgb_y_pred_fs, average = 'macro')
xgb_f1_score_fs = f1_score(y_test_encoded, xgb_y_pred_fs, average = 'macro')

print("======= Evaluation for XGBoost after FS =======")
print("Accuracy (After FS): ", xgb_accuracy_fs)
print("Precision (After FS): ", xgb_precision_fs)
print("Recall (After FS): ", xgb_recall_fs)
print("F1-Score (After FS): ", xgb_f1_score_fs)
print(classification_report(y_test_encoded, xgb_y_pred_fs, digits=4))

======= Evaluation for XGBoost after FS =======
Accuracy (After FS):  0.9608832807570978
Precision (After FS):  0.959021444076633
Recall (After FS):  0.9576666469565479
F1-Score (After FS):  0.958239833679
              precision    recall  f1-score   support

           0     0.9917    0.9835    0.9876       242
           1     0.9545    0.9921    0.9730       254
           2     0.8978    0.8789    0.8883       190
           3     0.9960    0.9920    0.9940       250
           4     0.9853    0.9571    0.9710       210
           5     0.9954    0.9954    0.9954       219
           6     0.8924    0.9045    0.8984       220

    accuracy                         0.9609      1585
   macro avg     0.9590    0.9577    0.9582      1585
weighted avg     0.9610    0.9609    0.9609      1585



# 3. LightGBM Classifier

In [49]:
import lightgbm as lgb

## 3.1. Train LightGBM before Feature Selection

In [50]:
lgb_model = lgb.LGBMClassifier()
lgb_model.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000837 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3263
[LightGBM] [Info] Number of data points in the train set: 6340, number of used features: 24
[LightGBM] [Info] Start training from score -1.921602
[LightGBM] [Info] Start training from score -1.925922
[LightGBM] [Info] Start training from score -1.963413
[LightGBM] [Info] Start training from score -1.913019
[LightGBM] [Info] Start training from score -2.044111
[LightGBM] [Info] Start training from score -1.921602
[LightGBM] [Info] Start training from score -1.937898


LGBMClassifier()

In [51]:
lgb_y_pred = lgb_model.predict(X_test)

In [52]:
lgb_accuracy = accuracy_score(y_test, lgb_y_pred)
lgb_precision = precision_score(y_test, lgb_y_pred, average = 'macro')
lgb_recall = recall_score(y_test, lgb_y_pred, average = 'macro')
lgb_f1_score = f1_score(y_test, lgb_y_pred, average = 'macro')

print("=========== Evaluation for LightGBM ===========")
print("Accuracy (Before FS): ", lgb_accuracy)
print("Precision (Before FS): ", lgb_precision)
print("Recall (Before FS): ", lgb_recall)
print("F1-Score (Before FS): ", lgb_f1_score)
print(classification_report(y_test, lgb_y_pred, digits=4))

=========== Evaluation for LightGBM ===========
Accuracy (Before FS):  0.9615141955835962
Precision (Before FS):  0.9589353665140123
Recall (Before FS):  0.9585418644977468
F1-Score (Before FS):  0.958667031662001
              precision    recall  f1-score   support

           1     0.9917    0.9876    0.9896       242
           2     0.9653    0.9843    0.9747       254
           3     0.8756    0.8895    0.8825       190
           4     0.9921    1.0000    0.9960       250
           5     0.9902    0.9667    0.9783       210
           6     0.9864    0.9954    0.9909       219
           7     0.9112    0.8864    0.8986       220

    accuracy                         0.9615      1585
   macro avg     0.9589    0.9585    0.9587      1585
weighted avg     0.9615    0.9615    0.9614      1585



## 3.2. Training LightGBM after Feature Selection

In [53]:
lgb_model_fs = lgb.LGBMClassifier()
lgb_model_fs.fit(X_train_fs, y_train_fs)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000084 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 514
[LightGBM] [Info] Number of data points in the train set: 6340, number of used features: 3
[LightGBM] [Info] Start training from score -1.921602
[LightGBM] [Info] Start training from score -1.925922
[LightGBM] [Info] Start training from score -1.963413
[LightGBM] [Info] Start training from score -1.913019
[LightGBM] [Info] Start training from score -2.044111
[LightGBM] [Info] Start training from score -1.921602
[LightGBM] [Info] Start training from score -1.937898


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


LGBMClassifier()

In [54]:
lgb_y_pred_fs = lgb_model_fs.predict(X_test_fs)

In [55]:
lgb_accuracy_fs = accuracy_score(y_test, lgb_y_pred_fs)
lgb_precision_fs = precision_score(y_test, lgb_y_pred_fs, average = 'macro')
lgb_recall_fs = recall_score(y_test, lgb_y_pred_fs, average = 'macro')
lgb_f1_score_fs = f1_score(y_test,lgb_y_pred_fs, average = 'macro')

print("======= Evaluation for XGBoost after FS =======")
print("Accuracy (After FS): ", lgb_accuracy_fs)
print("Precision (After FS): ", lgb_precision_fs)
print("Recall (After FS): ", lgb_recall_fs)
print("F1-Score (After FS): ", lgb_f1_score_fs)
print(classification_report(y_test, lgb_y_pred_fs, digits=4))

======= Evaluation for XGBoost after FS =======
Accuracy (After FS):  0.9627760252365931
Precision (After FS):  0.960471739661247
Recall (After FS):  0.9599244853294724
F1-Score (After FS):  0.9601580612754662
              precision    recall  f1-score   support

           1     0.9876    0.9835    0.9855       242
           2     0.9654    0.9882    0.9767       254
           3     0.8848    0.8895    0.8871       190
           4     0.9920    0.9960    0.9940       250
           5     0.9903    0.9714    0.9808       210
           6     1.0000    1.0000    1.0000       219
           7     0.9032    0.8909    0.8970       220

    accuracy                         0.9628      1585
   macro avg     0.9605    0.9599    0.9602      1585
weighted avg     0.9628    0.9628    0.9627      1585



## 4. Support Vector Machine Classifier

In [56]:
from sklearn import svm

## 4.1. Train SVM before Feature Selection

In [57]:
svm_model = svm.SVC()
svm_model.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC()

In [58]:
svm_y_pred = svm_model.predict(X_test)

In [59]:
svm_accuracy = accuracy_score(y_test, svm_y_pred)
svm_precision = precision_score(y_test, svm_y_pred, average = 'macro')
svm_recall = recall_score(y_test, svm_y_pred, average = 'macro')
svm_f1_score = f1_score(y_test, svm_y_pred, average = 'macro')

print("=========== Evaluation for SVM ===========")
print("Accuracy (Before FS): ", svm_accuracy)
print("Precision (Before FS): ", svm_precision)
print("Recall (Before FS): ", svm_recall)
print("F1-Score (Before FS): ", svm_f1_score)
print(classification_report(y_test, svm_y_pred, digits=4))

=========== Evaluation for SVM ===========
Accuracy (Before FS):  0.7274447949526814
Precision (Before FS):  0.7250303995534189
Recall (Before FS):  0.7145517769068063
F1-Score (Before FS):  0.7159995782469176
              precision    recall  f1-score   support

           1     0.8233    0.9050    0.8622       242
           2     0.7608    0.9016    0.8252       254
           3     0.4518    0.4684    0.4599       190
           4     0.9431    0.7960    0.8633       250
           5     0.8588    0.7238    0.7855       210
           6     0.6649    0.5616    0.6089       219
           7     0.5726    0.6455    0.6068       220

    accuracy                         0.7274      1585
   macro avg     0.7250    0.7146    0.7160      1585
weighted avg     0.7357    0.7274    0.7276      1585



## 4.2. Train SVM after Feature Selection

In [60]:
svm_model_fs = svm.SVC()
svm_model_fs.fit(X_train_fs, y_train)

svm_y_pred_fs = svm_model_fs.predict(X_test_fs)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [61]:
svm_accuracy_fs = accuracy_score(y_test, svm_y_pred_fs)
svm_precision_fs = precision_score(y_test, svm_y_pred_fs, average = 'macro')
svm_recall_fs = recall_score(y_test, svm_y_pred_fs, average = 'macro')
svm_f1_score_fs = f1_score(y_test,svm_y_pred_fs, average = 'macro')

print("======= Evaluation for SVM after FS =======")
print("Accuracy (After FS): ", svm_accuracy_fs)
print("Precision (After FS): ", svm_precision_fs)
print("Recall (After FS): ", svm_recall_fs)
print("F1-Score (After FS): ", svm_f1_score_fs)

print(classification_report(y_test, svm_y_pred_fs, digits=4))

======= Evaluation for SVM after FS =======
Accuracy (After FS):  0.8826498422712934
Precision (After FS):  0.8968021621792285
Recall (After FS):  0.8676620416010673
F1-Score (After FS):  0.8645616471320334
              precision    recall  f1-score   support

           1     1.0000    1.0000    1.0000       242
           2     0.7888    1.0000    0.8819       254
           3     0.8750    0.4421    0.5874       190
           4     1.0000    0.9480    0.9733       250
           5     1.0000    0.7381    0.8493       210
           6     0.7466    0.9954    0.8532       219
           7     0.8672    0.9500    0.9067       220

    accuracy                         0.8826      1585
   macro avg     0.8968    0.8677    0.8646      1585
weighted avg     0.8977    0.8826    0.8742      1585



## 5. Multilayered Perceptron

In [62]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

2025-08-08 13:20:26.210387: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754659226.396278      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754659226.456546      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [63]:
y_train_encoded = y_train - 1
y_test_encoded = y_test - 1
num_classes = len(np.unique(y_train_encoded))
y_train_cat = to_categorical(y_train_encoded, num_classes)
y_test_cat = to_categorical(y_test_encoded, num_classes)

## 5.1. MLP Classifier before Feature Selection

In [64]:
# Define architecture
mlp_model = Sequential()
mlp_model.add(Dense(128, input_shape=(X_train.shape[1],), activation='relu'))
mlp_model.add(Dropout(0.3))
mlp_model.add(Dense(64, activation='relu'))
mlp_model.add(Dropout(0.3))
mlp_model.add(Dense(num_classes, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1754659237.783416      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [65]:
mlp_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

mlp_model.fit(X_train, y_train_cat, epochs=100, batch_size=32, verbose=0)

I0000 00:00:1754659240.763842     125 service.cc:148] XLA service 0x7e3bd400c730 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1754659240.764417     125 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1754659240.992826     125 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1754659242.361419     125 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [66]:
mlp_y_pred_probs = mlp_model.predict(X_test)
mlp_y_pred = np.argmax(mlp_y_pred_probs, axis=1)
print("=== Evaluation Before Feature Selection ===")
print(classification_report(y_test_encoded, mlp_y_pred, digits=4))

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
=== Evaluation Before Feature Selection ===
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000       242
           1     0.9883    0.9961    0.9922       254
           2     0.8860    0.9000    0.8930       190
           3     1.0000    0.9960    0.9980       250
           4     0.9952    0.9905    0.9928       210
           5     0.9730    0.9863    0.9796       219
           6     0.9393    0.9136    0.9263       220

    accuracy                         0.9716      1585
   macro avg     0.9688    0.9689    0.9688      1585
weighted avg     0.9717    0.9716    0.9716      1585



## 5.2. MLP Model after Feature Selection

In [67]:
mlp_model_fs = Sequential()
mlp_model_fs.add(Dense(128, input_shape=(X_train_fs.shape[1],), activation='relu'))
mlp_model_fs.add(Dropout(0.3))
mlp_model_fs.add(Dense(64, activation='relu'))
mlp_model_fs.add(Dropout(0.3))
mlp_model_fs.add(Dense(num_classes, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [68]:
mlp_model_fs.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

mlp_model_fs.fit(X_train_fs, y_train_cat, epochs=100, batch_size=32, verbose=0)

In [69]:
mlp_y_pred_fs_probs = mlp_model_fs.predict(X_test_fs)
mlp_y_pred_fs = np.argmax(mlp_y_pred_fs_probs, axis=1)
print("=== Evaluation After Feature Selection ===")
print(classification_report(y_test_encoded, mlp_y_pred_fs, digits=4))

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
=== Evaluation After Feature Selection ===
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000       242
           1     1.0000    1.0000    1.0000       254
           2     1.0000    0.7789    0.8757       190
           3     1.0000    1.0000    1.0000       250
           4     1.0000    1.0000    1.0000       210
           5     1.0000    1.0000    1.0000       219
           6     0.8397    1.0000    0.9129       220

    accuracy                         0.9735      1585
   macro avg     0.9771    0.9684    0.9698      1585
weighted avg     0.9777    0.9735    0.9730      1585

